In [25]:
import pandas as pd
import psycopg2
import sqlite3
from sklearn.linear_model import LogisticRegression

equipment_data = pd.read_csv('equipment_sensor.csv')
maintenance_data = pd.read_csv('maintenance_records.csv')
network_data = pd.read_csv('network_sensor.csv')

In [ ]:
#Removing duplicates and fixing missing data
equipment_data.drop_duplicates(inplace=True)
equipment_data.fillna(method='ffill', inplace=True)

maintenance_data.drop_duplicates(inplace=True)
maintenance_data.fillna(method='ffill', inplace=True)

network_data.drop_duplicates(inplace=True)
network_data.fillna(method='ffill', inplace=True)



In [ ]:
# Normalizing data
equipment_data['date'] = pd.to_datetime(equipment_data['date'])
network_data['date'] = pd.to_datetime(network_data['date'])
maintenance_data['date'] = pd.to_datetime(maintenance_data['date'])

In [ ]:
# Aggregating data
equipment_failures = equipment_data.groupby('ID').count()['sensor_reading']
average_time_between_failures = equipment_data.groupby('ID')['date'].apply(lambda x: x.diff().mean())


In [ ]:
# Joining data
equipment_maintenance = pd.merge(equipment_data, maintenance_data, on='ID', how='left')
equipment_maintenance

,ID,date_x,time_x,sensor_reading,date_y,time_y,equipment_ID,maintenance_type
0,1,2022-03-01,8:00:00,26.7,2022-03-01,10:00:00,1,Preventive Maintenance
1,1,2022-03-01,8:15:00,28.4,2022-03-01,10:00:00,1,Preventive Maintenance
2,1,2022-03-01,8:30:00,27.8,2022-03-01,10:00:00,1,Preventive Maintenance
3,2,2022-03-01,8:00:00,99.1,2022-03-02,14:30:00,2,Corrective Maintenance
4,2,2022-03-01,8:15:00,97.5,2022-03-02,14:30:00,2,Corrective Maintenance
5,2,2022-03-01,8:30:00,98.2,2022-03-02,14:30:00,2,Corrective Maintenance


In [27]:
print(equipment_maintenance.columns)

Index(['ID', 'date_x', 'time_x', 'sensor_reading', 'date_y', 'time_y',
       'equipment_ID', 'maintenance_type'],
      dtype='object')


In [ ]:
# Getting the total number of equipment failures and the average time between failures:
equipment_data['time'] = pd.to_datetime(equipment_data['time'])
num_failures = equipment_data[equipment_data['sensor_reading'] == 'failure'].groupby('ID').count()
avg_time_between_failures = equipment_data.groupby('ID')['time'].diff().mean()
print(avg_time_between_failures) 

0 days 00:15:00


In [19]:
!curl ipecho.net/plain

34.73.18.189

In [20]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
conn = psycopg2.connect(
    host="34.93.255.71",
    database="Postgres_db",
    user="students1",
    password="Test12"
)

In [ ]:
# establish a connection to the database
conn = sqlite3.connect('Postgres_db')

# create a cursor object
cur = conn.cursor()

# create a table for the data
cur.execute("""
    CREATE TABLE IF NOT EXISTS equipment_maintenance (
        id INTEGER PRIMARY KEY,
        equipment_id INTEGER,
        date_x DATE,
        maintenance_type TEXT
    );
""")

# insert the data into the table
equipment_maintenance.to_sql(name="equipment_maintenance", con=conn, if_exists="append", index=False)

# commit the changes to the database
conn.commit()

# close the cursor and the connection
cur.close()
conn.close()